In [5]:
import tensorflow as tf
from tensorflow import keras

In [13]:

W = tf.Variable(tf.ones(shape=(2,2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")

@tf.function
def simple_model(x):
    return W * x + b
out = simple_model([1,0])
print(out)

tf.Tensor(
[[1. 0.]
 [1. 0.]], shape=(2, 2), dtype=float32)


In [11]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [70]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

print(np.shape(train_images))

(1000, 784)


In [14]:
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels)) 

Epoch 1/10
32/32 [==============================] - 0s 6ms/step - loss: 1.1482 - sparse_categorical_accuracy: 0.6630 - val_loss: 0.7143 - val_sparse_categorical_accuracy: 0.7830
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.4206 - sparse_categorical_accuracy: 0.8960 - val_loss: 0.5610 - val_sparse_categorical_accuracy: 0.8170
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2838 - sparse_categorical_accuracy: 0.9260 - val_loss: 0.5029 - val_sparse_categorical_accuracy: 0.8350
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2126 - sparse_categorical_accuracy: 0.9430 - val_loss: 0.4319 - val_sparse_categorical_accuracy: 0.8610
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1496 - sparse_categorical_accuracy: 0.9700 - val_loss: 0.4252 - val_sparse_categorical_accuracy: 0.8580
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1163 - sparse_categorical_accuracy: 

In [15]:
# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))
model.save("models/mnist/1/model.savedmodel")

32/32 - 0s - loss: 0.4195 - sparse_categorical_accuracy: 0.8690
Untrained model, accuracy: 86.90%
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/mnist/1/model.savedmodel/assets


In [89]:
model.predict(test_images[0:1])

array([[ 0.17135115, -6.4116397 , -0.34636346,  1.4378225 , -4.5923405 ,
        -1.3223665 , -8.255523  , 10.684266  , -2.4663098 ,  2.3591187 ]],
      dtype=float32)

### Triton Server Setup

In [ ]:
!docker run --rm -p 8000:8000 -p 8001:8001 -p 8002:8002 -v /home/sachin/Documents/OXFORD/models:/models nvcr.io/nvidia/tritonserver:20.11-py3 tritonserver --model-repository=/models

In [62]:
import tritonclient.http as httpclient
triton_client = httpclient.InferenceServerClient(url="localhost:8000")
triton_client.get_model_metadata("mnist")

{'name': 'mnist',
 'versions': ['1'],
 'platform': 'tensorflow_savedmodel',
 'inputs': [{'name': 'dense_input', 'datatype': 'FP32', 'shape': [-1, 784]}],
 'outputs': [{'name': 'dense_1', 'datatype': 'FP32', 'shape': [-1, 10]}]}

In [86]:
import numpy as np
inputs = []
outputs = []
inputs.append(httpclient.InferInput('dense_input', [1,784], "FP32"))

inputs[0].set_data_from_numpy(np.float32(test_images[0:1]))
outputs.append(httpclient.InferRequestedOutput('dense_1',binary_data=False))
results = triton_client.infer("mnist",inputs,outputs=outputs)

print(results.get_response())

{'model_name': 'mnist', 'model_version': '1', 'outputs': [{'name': 'dense_1', 'datatype': 'FP32', 'shape': [1, 10], 'data': [0.17135114967823029, -6.41163969039917, -0.34636345505714417, 1.4378224611282349, -4.592340469360352, -1.32236647605896, -8.255522727966309, 10.684266090393066, -2.4663097858428955, 2.359118700027466]}]}
